In [ ]:
from pymongo import MongoClient
import pymongo

from tqdm.notebook import tqdm
import networkx as nx

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

import pickle

In [ ]:
with open('data/dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
data = []
g_train = dataset['train']
for u in tqdm(dataset['users'].values()):
    for s in g_train[u]:
        data.append([u, s, g_train.edges()[(u, s)]['scrobbles']])
 
df = pd.DataFrame(data=data, columns=['user', 'track', 'scrobbles'])

  0%|          | 0/3307 [00:00<?, ?it/s]

In [ ]:
df.describe()

,user,track,scrobbles
count,2.564908e+06,2.564908e+06,2.564908e+06
mean,1.464116e+03,1.110688e+05,4.445133e+01
std,9.616420e+02,7.791742e+04,2.842347e+02
min,0.000000e+00,3.307000e+03,1.000000e+00
25%,6.240000e+02,1.405600e+04,8.000000e+00
50%,1.472000e+03,1.312660e+05,2.100000e+01
75%,2.227000e+03,1.712930e+05,4.500000e+01
max,3.306000e+03,2.553200e+05,2.056890e+05


In [ ]:
def search_superior(data, percentage=0.01, dmin=None, dmax=None):
    total = data.shape[0]
    if dmin is None:
        dmin = np.min(data)
    if dmax is None:
        dmax = np.max(data)
    mid = (dmax + dmin) // 2
    while dmin != mid and mid != dmax:
        p = np.sum(data >= mid) / total
        #print(f'{p} {dmin} {mid} {dmax}')
        if p > percentage:
            dmin = mid
        elif p < percentage:
            dmax = mid
        else:
            return mid
        mid = (dmax + dmin) // 2
    pmin = np.sum(data > dmin) / total
    pmax = np.sum(data > dmax) / total
    if abs(pmin - percentage) < abs(pmax - percentage):
        return dmin
    return dmax

print(search_superior(df['scrobbles'].values, percentage=0.01))
print(search_superior(df['scrobbles'].values, percentage=0.02))
print(search_superior(df['scrobbles'].values, percentage=0.03))
df['scrobbles'] = np.clip(df['scrobbles'], 0, search_superior(df['scrobbles'].values, percentage=0.03))

374
255
202


In [ ]:
df.describe()

,user,track,scrobbles
count,2.564908e+06,2.564908e+06,2.564908e+06
mean,1.464116e+03,1.110688e+05,3.696133e+01
std,9.616420e+02,7.791742e+04,4.584949e+01
min,0.000000e+00,3.307000e+03,1.000000e+00
25%,6.240000e+02,1.405600e+04,8.000000e+00
50%,1.472000e+03,1.312660e+05,2.100000e+01
75%,2.227000e+03,1.712930e+05,4.500000e+01
max,3.306000e+03,2.553200e+05,2.020000e+02


# Embeddings

In [ ]:
x1 = np.asarray(df['user'], dtype=np.int32)[:, np.newaxis]
x2 = np.asarray(df['track'], dtype=np.int32)[:, np.newaxis] - len(dataset['users'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, Add, Lambda, Concatenate, Activation
from tensorflow.keras.models import Model
import functools
import tensorflow.keras.backend as K
from scipy.sparse import dok_matrix
import random


seed_value = 42
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)

In [ ]:
def base_model(users, tracks, vector_dim=64):
    """
    A Keras implementation of the GloVe architecture
    :param vocab_size: The number of distinct words
    :param vector_dim: The vector dimension of each word
    :return:
    """
    input_target = Input((1,), name='user_id')
    input_context = Input((1,), name='track_id')

    central_embedding = Embedding(users, vector_dim, input_length=1, name='user_emb')

    context_embedding = Embedding(tracks, vector_dim, input_length=1, name='track_emb')

    vector_target = central_embedding(input_target)
    vector_context = context_embedding(input_context)

    dot_product = Dot(axes=-1)([vector_target, vector_context])
    prediction = Reshape((1, ))(dot_product)
    prediction = Activation('sigmoid')(prediction)

    model = Model(inputs=[input_target, input_context], outputs=prediction)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    model.summary()

    return model

In [ ]:
model = base_model(len(dataset['users']), len(dataset['train'].nodes())-len(dataset['users']), 10)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
track_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_emb (Embedding)            (None, 1, 10)        33070       user_id[0][0]                    
__________________________________________________________________________________________________
track_emb (Embedding)           (None, 1, 10)        2520140     track_id[0][0]                   
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
import math


class NegSamping(Sequence):
    
    def __init__(self, users, tracks, n_users, n_tracks, batch_size):
        self.users = users
        self.tracks = tracks
        self.n_users = n_users
        self.n_tracks = n_tracks
        self.batch_size = batch_size
        self.on_epoch_end()
    
    def __len__(self):
        return math.ceil(self.tracks.shape[0] / self.batch_size)
    
    def __getitem__(self, idx):
        x1 = self.users[idx * self.batch_size:(idx+1) * self.batch_size:, ...]
        x2 = self.tracks[idx * self.batch_size:(idx+1) * self.batch_size:, ...]
        y = np.ones((x1.shape[0], 1))
        x1_n = np.random.randint(0, self.n_users, size=x1.shape)
        x2_n = np.random.randint(0, self.n_tracks, size=x2.shape)
        y_n = np.zeros_like(y)
        return [np.concatenate((x1, x1_n), axis=0),
                np.concatenate((x2, x2_n), axis=0)], np.concatenate((y, y_n), axis=0)
    
    def on_epoch_end(self):
        self.users, self.tracks = shuffle(self.users, self.tracks)

In [ ]:
ds = NegSamping(x1, x2, len(dataset['users']),len(dataset['train'].nodes())-len(dataset['users']), 500)

In [ ]:
ds[0]

([array([[1363],
         [3160],
         [ 146],
         [2285],
         [ 321],
         [ 122],
         [ 247],
         [ 170],
         [ 447],
         [ 955],
         [ 127],
         [2303],
         [3056],
         [  37],
         [1680],
         [1671],
         [1291],
         [ 192],
         [1602],
         [   3],
         [  79],
         [ 450],
         [ 718],
         [1113],
         [1082],
         [ 100],
         [2778],
         [  27],
         [   6],
         [   5],
         [1914],
         [1388],
         [ 413],
         [1676],
         [1130],
         [   4],
         [1491],
         [1041],
         [1127],
         [1588],
         [1449],
         [1176],
         [   0],
         [2767],
         [ 116],
         [1068],
         [  11],
         [2776],
         [1678],
         [1632],
         [ 795],
         [1135],
         [2232],
         [2508],
         [3215],
         [ 827],
         [3108],
         [3228],
         [2449

In [ ]:
if not os.path.exists('data/embeddedings.h5'):
    model.fit(NegSamping(x1, x2,
                         len(dataset['users']), 
                         len(dataset['train'].nodes())-len(dataset['users']),
                         500), epochs=15) 
    model.save_weights('data/embeddedings.h5')
else:
    model.load_weights('data/embeddedings.h5')

  ...
    to  
  ['...']
Train for 5130 steps
Epoch 1/15
5130/5130 [==============================] - 211s 41ms/step - loss: 0.6237 - binary_accuracy: 0.6473
Epoch 2/15
5130/5130 [==============================] - 129s 25ms/step - loss: 0.5297 - binary_accuracy: 0.7401
Epoch 3/15
5130/5130 [==============================] - 120s 23ms/step - loss: 0.4835 - binary_accuracy: 0.7722
Epoch 4/15
5130/5130 [==============================] - 117s 23ms/step - loss: 0.4378 - binary_accuracy: 0.7976
Epoch 5/15
5130/5130 [==============================] - 118s 23ms/step - loss: 0.3968 - binary_accuracy: 0.8220
Epoch 6/15
5130/5130 [==============================] - 133s 26ms/step - loss: 0.3637 - binary_accuracy: 0.8409
Epoch 7/15
5130/5130 [==============================] - 127s 25ms/step - loss: 0.3383 - binary_accuracy: 0.8546
Epoch 8/15
5130/5130 [==============================] - 124s 24ms/step - loss: 0.3183 - binary_accuracy: 0.8651
Epoch 9/15
5130/5130 [==============================] - 11

In [ ]:
print(model.layers[2].name)
users_embs = K.get_value(model.layers[2].embeddings)
print(model.layers[3].name)
tracks_embs = K.get_value(model.layers[3].embeddings)

user_emb
track_emb


In [ ]:
model.predict([x1, x2], batch_size=10000)

array([[0.97153294],
       [0.9502234 ],
       [0.96148473],
       ...,
       [0.95455116],
       [0.97605115],
       [0.7787171 ]], dtype=float32)

In [ ]:
del model

# Cosines

In [ ]:
if not os.path.exists('data/cos.npz'):
    len_users = np.linalg.norm(users_embs, axis=1)[:, np.newaxis]
    len_tracks = np.linalg.norm(tracks_embs, axis=1)[:, np.newaxis]

    cos = (users_embs @ tracks_embs.T) / (len_users @ len_tracks.T)
    np.savez_compressed('data/cos.npz', cosines=cos)
else:
    cos = np.load('data/cos.npz')['cosines']
del users_embs
del tracks_embs

In [ ]:
print(cos)
print(np.min(cos))
print(np.max(cos))
print(np.mean(cos))
print(np.std(cos))

[[ 0.6166293   0.7904514   0.73205227 ...  0.5750937   0.23726128
   0.61921895]
 [ 0.29186314  0.22068222  0.24940278 ...  0.5279246   0.18231632
   0.74171233]
 [ 0.11913287  0.07204851 -0.15346918 ... -0.474571   -0.3752259
  -0.35964486]
 ...
 [ 0.3360403   0.24041389  0.23741494 ... -0.64983857 -0.63236153
  -0.4218958 ]
 [ 0.04058007 -0.28635052 -0.17672122 ... -0.4882339  -0.43182966
  -0.5769507 ]
 [ 0.09148319 -0.1621145  -0.21006835 ... -0.7100018  -0.7428562
  -0.6279147 ]]
-0.99545115
0.98095804
-0.32127735
0.26847863


# Communities test

In [ ]:
from networkx.algorithms.community import greedy_modularity_communities

In [ ]:
if not os.path.exists('data/commutities.pickle'):
    communities = list(greedy_modularity_communities(dataset['train']))
    with open('data/commutities.pickle', 'wb') as f:
        pickle.dump(communities, f)
else:
    with open('data/commutities.pickle', 'rb') as f:
        communities = pickle.load(f)

In [ ]:
type(communities)

In [ ]:
node_community = {}
for c, group in enumerate(communities):
    for u in group:
        node_community[u] = c

In [ ]:
from collections import deque
c_sim = 0
c_diff = 0
c_all_cos = 0

for i in tqdm(range(len(dataset['users']))):
    for j in range(len(dataset['users']), len(dataset['train'].nodes())):
        c_all_cos += 1
        if node_community[i] == node_community[j]:
            c_sim += 1
        else:
            c_diff += 1
            
sim = np.empty((c_sim,))
diff = np.empty((c_diff,))
all_cos = np.empty((c_all_cos,))
i_sim = 0
i_diff = 0
i_all_cos = 0

for i in tqdm(range(len(dataset['users']))):
    for j in range(len(dataset['users']), len(dataset['train'].nodes())):
        current_cos = cos[i, j - len(dataset['users'])]
        all_cos[i_all_cos] = current_cos
        i_all_cos += 1
        if node_community[i] == node_community[j]:
            sim[i_sim] = current_cos
            i_sim += 1
        else:
            diff[i_diff] = current_cos
            i_diff += 1
            
print(c_diff == i_diff)
print(c_sim == i_sim)
print(c_all_cos == i_all_cos)

In [ ]:
print(len(all_cos))
print(np.mean(all_cos))
print(np.std(all_cos))
print(len(sim))
print(np.mean(sim))
print(np.std(sim))
print(len(diff))
print(np.mean(diff))
print(np.std(diff))

In [ ]:
from scipy.stats import ttest_ind

ttest_ind(sim, diff, alternative='greater', equal_var=False)

In [ ]:
ttest_ind(sim, all_cos, alternative='greater', equal_var=False)

In [ ]:
ttest_ind(all_cos, diff, alternative='greater', equal_var=False)